# Module 14 — Contacts API Walkthrough

Цей notebook дозволяє покроково перевірити кожен endpoint API.  
Одна клітина = один крок. Токен зберігається між клітинами автоматично.

**Передумова:** запустити docker-compose перед початком
```bash
cd module_14/contacts_api
docker-compose up -d
```

| Сервіс | URL |
|--------|-----|
| API (Swagger) | http://localhost:8000/docs |
| Web UI | http://localhost:8000/app/login.html |
| MailHog | http://localhost:8025 |

---

## Cell 0 — Setup

Запускаємо один раз. Всі наступні клітини використовують ці змінні.

In [1]:
import httpx
import json
import time

BASE = "http://localhost:8000"

# Змінні стану — заповнюються по ходу виконання клітин
access_token = None
refresh_token = None
contact_id = None

def pretty(r):
    """Вивести статус + JSON відповідь красиво."""
    print(f"HTTP {r.status_code}")
    try:
        print(json.dumps(r.json(), indent=2, ensure_ascii=False))
    except Exception:
        print(r.text or "(no body)")

print("✅ Setup готовий")

✅ Setup готовий


---
## Блок 1 — Auth
### 1.1 Health Check

Перевіряємо що API запущений.

In [4]:
r = httpx.get(f"{BASE}/healthz")
pretty(r)
# Очікуємо: HTTP 200  {"status": "ok"}

ReadTimeout: timed out

### 1.2 Register

`POST /api/v1/auth/register` — створює акаунт і надсилає лист у MailHog.  
Статус `201 Created`, у відповіді — об'єкт User (без пароля).

In [ ]:
r = httpx.post(f"{BASE}/api/v1/auth/register", json={
    "email": "demo@example.com",
    "password": "Secret123!"
})
pretty(r)
# 201 → акаунт створено, перевір http://localhost:8025
# 409 → email вже зареєстровано

### 1.3 Verify Email

1. Відкрий **http://localhost:8025** (MailHog)
2. Знайди лист з темою "Verify your email"
3. Скопіюй токен з посилання (частина після `/verify/`)
4. Вставте нижче у `email_token = "..."`

> **Чому окремий токен?** Access token не підходить — він має `purpose: "access"`.  
> Email token має `purpose: "email_verify"` + TTL 24 год. Це захист від підміни.

In [ ]:
email_token = "PASTE_TOKEN_FROM_MAILHOG_HERE"

r = httpx.get(f"{BASE}/api/v1/auth/verify/{email_token}")
pretty(r)
# 200 → {"message": "Email verified. You can now log in."}
# 400 → токен протермінований або невалідний

### 1.4 Login

`POST /api/v1/auth/login` — повертає `access_token` (30 хв) і `refresh_token` (7 днів).

> **Важливо:** цей endpoint приймає **form-data** (не JSON), бо реалізує `OAuth2PasswordRequestForm`.  
> Браузер і Postman типово відправляють `application/x-www-form-urlencoded`.

In [ ]:
r = httpx.post(f"{BASE}/api/v1/auth/login", data={
    "username": "demo@example.com",  # поле зветься username (OAuth2 стандарт)
    "password": "Secret123!"
})
pretty(r)

if r.status_code == 200:
    tokens = r.json()
    access_token = tokens["access_token"]
    refresh_token = tokens["refresh_token"]
    print(f"\naccess_token збережено: {access_token[:50]}...")

### 1.5 Login без верифікації (403 demo)

Що буде якщо спробувати увійти з непідтвердженим email?  
API повертає `403 Forbidden` (а не `401`) — щоб клієнт міг показати правильне повідомлення.

In [ ]:
# Реєструємо нового юзера
httpx.post(f"{BASE}/api/v1/auth/register", json={
    "email": "unverified@example.com",
    "password": "Test123!"
})

# Одразу намагаємось залогінитись — без верифікації
r = httpx.post(f"{BASE}/api/v1/auth/login", data={
    "username": "unverified@example.com",
    "password": "Test123!"
})
pretty(r)
# 403 Forbidden — "Email address is not verified. Check your inbox."

### 1.6 Refresh Token

Access token протерміновується за 30 хвилин. Клієнт використовує refresh token щоб отримати новий — без повторного логіну.

In [ ]:
r = httpx.post(f"{BASE}/api/v1/auth/refresh", json={
    "refresh_token": refresh_token
})
pretty(r)

if r.status_code == 200:
    access_token = r.json()["access_token"]
    print("\nNew access_token отримано ✓")

---
## Блок 2 — User Profile
### 2.1 Мій профіль

In [ ]:
auth_headers = {"Authorization": f"Bearer {access_token}"}

r = httpx.get(f"{BASE}/api/v1/users/me", headers=auth_headers)
pretty(r)
# {"id": 1, "email": "...", "is_verified": true, "avatar_url": null}

---
## Блок 3 — Contacts CRUD
### 3.1 Створити контакт

In [ ]:
r = httpx.post(f"{BASE}/api/v1/contacts/", headers=auth_headers, json={
    "first_name": "Іван",
    "last_name": "Франко",
    "email": "ivan@example.com",
    "phone": "+380501234567",
    "birthday": "1856-08-27"
})
pretty(r)
# 201 Created

if r.status_code == 201:
    contact_id = r.json()["id"]
    print(f"\ncontact_id = {contact_id}")

### 3.2 Список контактів

In [ ]:
r = httpx.get(f"{BASE}/api/v1/contacts/", headers=auth_headers)
contacts = r.json()
print(f"HTTP {r.status_code} — всього: {len(contacts)}")
for c in contacts:
    print(f"  [{c['id']}] {c['first_name']} {c['last_name']} | {c.get('email', '—')}")

### 3.3 Пошук

In [ ]:
r = httpx.get(f"{BASE}/api/v1/contacts/", headers=auth_headers,
              params={"search": "Іван"})
pretty(r)
# Фільтрація по first_name, last_name, email

### 3.4 Один контакт

In [ ]:
r = httpx.get(f"{BASE}/api/v1/contacts/{contact_id}", headers=auth_headers)
pretty(r)

### 3.5 PATCH — часткове оновлення

Тільки передані поля змінюються. Решта залишається без змін.

In [ ]:
r = httpx.patch(f"{BASE}/api/v1/contacts/{contact_id}",
                headers=auth_headers,
                json={"phone": "+380999999999"})
pretty(r)
# Тільки phone змінився, birthday залишився 1856-08-27

### 3.6 PUT — повна заміна

Всі поля замінюються. Відсутні поля → null (або default).

In [ ]:
r = httpx.put(f"{BASE}/api/v1/contacts/{contact_id}",
              headers=auth_headers,
              json={
                  "first_name": "Іван",
                  "last_name": "Франко",
                  "email": "ivan.franko@example.com",
                  "phone": "+380501234567",
                  "birthday": "1856-08-27"
              })
pretty(r)
print(f"\nEmail оновлено: {r.json().get('email')}")

### 3.7 Birthdays — Redis cache demo

Перший виклик — запит до PostgreSQL.  
Другий виклик (протягом 1 год.) — відповідь із Redis кешу.  
Перевіряємо різницю у часі відповіді.

> Ключ кешу: `birthdays:{user_id}:{today}` — автоматично інвалідується наступного дня.

In [ ]:
# Спочатку додаємо контакт з найближчим днем народження
from datetime import date, timedelta
soon = (date.today() + timedelta(days=3)).isoformat()

httpx.post(f"{BASE}/api/v1/contacts/", headers=auth_headers, json={
    "first_name": "Тест",
    "last_name": "Юбіляр",
    "birthday": soon
})

# 1-й виклик — DB hit
t0 = time.perf_counter()
r1 = httpx.get(f"{BASE}/api/v1/contacts/birthdays", headers=auth_headers)
t1 = time.perf_counter()

# 2-й виклик — з Redis
r2 = httpx.get(f"{BASE}/api/v1/contacts/birthdays", headers=auth_headers)
t2 = time.perf_counter()

print(f"1-й виклик (DB):    {(t1-t0)*1000:.1f} мс")
print(f"2-й виклик (cache): {(t2-t1)*1000:.1f} мс")
print(f"\nРезультат: {json.dumps(r1.json(), indent=2, ensure_ascii=False)}")

### 3.8 Видалити контакт

`DELETE` повертає `204 No Content` — тіло відповіді порожнє.

In [ ]:
r = httpx.delete(f"{BASE}/api/v1/contacts/{contact_id}", headers=auth_headers)
print(f"HTTP {r.status_code}")  # 204

# Перевіряємо — контакт зник
r2 = httpx.get(f"{BASE}/api/v1/contacts/{contact_id}", headers=auth_headers)
print(f"GET після DELETE: HTTP {r2.status_code}")  # 404

---
## Блок 4 — Rate Limiting
### 4.1 Демонстрація 429

`POST /login` має ліміт **10 запитів на хвилину** (SlowAPI + Redis).  
Після перевищення — `429 Too Many Requests`.

In [ ]:
print("Надсилаємо 12 запитів login...\n")

for i in range(1, 13):
    r = httpx.post(f"{BASE}/api/v1/auth/login",
                   data={"username": "x@x.com", "password": "wrong"})
    status = r.status_code
    mark = "⛔" if status == 429 else "  "
    print(f"  {mark} [{i:02d}] HTTP {status}")
    if status == 429:
        print(f"         → {r.json().get('detail', '')}")
        break

# SlowAPI використовує IP адресу як ключ + Redis для зберігання лічильника

---
## Блок 5 — Logout & Token Blacklist
### 5.1 Logout

JWT токен не можна видалити (він stateless). Замість цього:  
1. Refresh token заноситься у Redis blacklist  
2. При наступному `/refresh` — перевіряємо blacklist → `401`

In [ ]:
# Logout — занести refresh_token у Redis blacklist
r = httpx.post(f"{BASE}/api/v1/auth/logout",
               json={"refresh_token": refresh_token})
print(f"Logout: HTTP {r.status_code}")  # 204 No Content

# Спроба використати той самий refresh_token знову
r2 = httpx.post(f"{BASE}/api/v1/auth/refresh",
                json={"refresh_token": refresh_token})
print(f"\nRefresh після logout: HTTP {r2.status_code}")  # 401
pretty(r2)

---
## Підсумок

| Endpoint | Метод | Особливості |
|----------|-------|-------------|
| `/auth/register` | POST | Rate limit 5/min, надсилає email |
| `/auth/verify/{token}` | GET | JWT з purpose=email_verify |
| `/auth/login` | POST | form-data, rate limit 10/min, 403 якщо не verified |
| `/auth/refresh` | POST | Перевіряє Redis blacklist |
| `/auth/logout` | POST | Заносить refresh token у Redis, 204 |
| `/users/me` | GET | Потребує Bearer token |
| `/contacts/` | GET | Пошук + пагінація |
| `/contacts/` | POST | Інвалідує cache після створення |
| `/contacts/birthdays` | GET | Redis cache-aside, TTL 1 год |
| `/contacts/{id}` | PATCH | Часткове оновлення |
| `/contacts/{id}` | DELETE | 204, інвалідує cache |

**Web UI:** http://localhost:8000/app/login.html